In [1]:
import pandas as pd
import os

In [2]:
data_path = "data/raw"
Ratebeer_path = lambda path: f"{path}/Ratebeer"
BeerAdvocate_path = lambda path: f"{path}/BeerAdvocate"
Matched_path = lambda path: f"{path}/MatchedBeerData"
clean_path = "data/clean"

In [3]:
common_replacements = {
    "Ã¡": "á", "Ã­": "í", "Ãº": "ú",
    "Ã±": "ñ", "Ã": "Ñ", "Ã¼": "ü", "Ã": "Ü",
    "Ã€": "À", "Ã©": "é", "Ã³": "ó", "Ã": "Í", "â": "'", "Ã": "ß", "": "'",
}

def replace_common_chars(string):
    for key, value in common_replacements.items():
        string = string.replace(key, value)
    return string

def replace_common_enc_errors(df):
    for column in string_columns(df):
        df[column] = df[column].apply(replace_common_chars)
    return df

def decode_string(column):
    """
    Decodes a column with weird characters like \x92\x09, they converted 
    are utf-8 equivalent for readability, it is not perfect but most common 
    characters are converted
    """
    return column[~column.isna()].apply(
        lambda x: replace_common_chars(x.encode().decode('unicode_escape').encode('latin1').decode('utf-8')))

def get_us_states(df):
    return df[df["location"].str.contains(",")]["location"].unique()

def filter_by_locations(df, locations):
    return df[df["location"].isin(locations)]

def filter_by_state(df, state):
    return df[df["location"].str.contains(state)]

def filter_only_americans(df):
    return filter_by_locations(df, get_us_states(df))

def filter_by_users(df, users):
    return df[df["user_id"].isin(users["user_id"])]

def extract_nan_as_column(df, column_name, replace_value=0):
    """
    If a column has nans it extracts them from the column and creates a new column with the same name and '_nan' appended. Nan values are replaced by the replace_value and the new column has 1 where the original column had nan.
    
    For logistic regression, this is useful to let the model decide how best to replace nan values
    """
    if df[column_name].isna().sum() == 0:
        return
    nan_column_name = f"{column_name}_nan"
    is_nan = df[column_name].isna()
    df[column_name][is_nan] = replace_value
    df[nan_column_name] = is_nan.astype(int)
    
def decode_into_utf8_df(df):
    string_columns = df.dtypes[ratings_ba.dtypes == "object"].index
    for column in string_columns:
        df[column] = decode_string(df[column])
   
   
def numerical_columns(df):
    return df.dtypes[df.dtypes != "object"].index[1:]

def string_columns(df):
    return df.dtypes[df.dtypes == "object"].index

def extract_nans_as_columns_df(df, column_names, replace_value=0):
    for column in column_names:
        extract_nan_as_column(df, column, replace_value=replace_value)
        
def clean_df(df, clean_nan_func, clean_chars_func):
    """
    Cleans a dataframe by cleaning all string columns and extracting nans as columns for columns with nans
    """
    for column in string_columns(df):
        df[column] = clean_chars_func(df[column], clean_nan_func, clean_chars_func)
    extract_nans_as_columns_df(df, string_columns(df), replace_value="")
    extract_nans_as_columns_df(df, numerical_columns(df), replace_value=0)

def clean_ratings(df):
    """
    Cleans a dataframe by cleaning all string columns and extracting nans as columns for columns with nans
    """
    for column in string_columns(df):
        df[column] = decode_string(df[column])
    extract_nans_as_columns_df(df, string_columns(df), replace_value="")
    extract_nans_as_columns_df(df, numerical_columns(df), replace_value=0)
    return df

def clean_matched_data(df):
    """
    Basic cleaning that is common to all matched data:
    - Replace column names with right ones (e.g. 'ba_1' -> 'ba_user_id')
    - Delete the first row as it contains the column names in the original dataset
    - Convert columns to numeric if possible
    - Extract nans as columns for numerical columns
    """
    # Since the first row is the name of the columns, we drop it and change the column names to lowercase
    df.columns = df.columns.str.lower()
    first_rows = df.iloc[0]
    # Rename columns so we can remove the first row
    df.columns = [column if i == 0 else f"{column.split('.')[0]}_{first_rows[column]}" for i, column in
                  enumerate(df.columns)]
    df = df.iloc[1:]
    df = df.apply(pd.to_numeric, errors='ignore')
    extract_nans_as_columns_df(df, numerical_columns(df), replace_value=0)
    extract_nans_as_columns_df(df, string_columns(df), replace_value="")
    return df

## Clean BeerAdvocate data

In [4]:
users_ba = pd.read_csv(BeerAdvocate_path(data_path) + "/BeerAdvocate_users.csv")

**Clean BeerAdvocate users.csv**

In [5]:
us_users_ba = filter_only_americans(users_ba.dropna(subset=["user_name", "location"]))
extract_nans_as_columns_df(us_users_ba, numerical_columns(us_users_ba), replace_value=0)
extract_nans_as_columns_df(us_users_ba, string_columns(us_users_ba), replace_value="")

Save the cleaned american users to a csv file

In [6]:
os.makedirs(BeerAdvocate_path(clean_path), exist_ok=True)
with open(BeerAdvocate_path(clean_path) + "/usa_users.csv", "w") as f:
    us_users_ba.to_csv(f, index=False)

### Clean ratings

- Keep only reviews originating from the US
- Drop rows with missing user_id or beer_id
- Decode all string columns
- Extract nans as seperate column for numerical columns, replacing nans with 0 in the original column
- Extract nans as seperate columns for string columns, replacing nans with "" in the original column

In [7]:
ratings_ba = pd.read_csv(BeerAdvocate_path(data_path) + "/BeerAdvocate_ratings.csv")

In [8]:
ratings_ba = ratings_ba.dropna(subset=["user_id", "beer_id"])
ratings_ba = clean_ratings(ratings_ba)
us_ratings_ba = filter_by_users(ratings_ba, us_users_ba)

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[column_name][is_nan] = replace_value
/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183

In [9]:
with open(BeerAdvocate_path(clean_path) + "/usa_ratings.csv", "w") as f:
    us_ratings_ba.to_csv(f, index=False)
    
del ratings_ba
del us_ratings_ba
del users_ba
del us_users_ba

### Clean BeerAdvocate beers.csv

- Extract nans
- replace common encoding errors

In [11]:
beer_ba = pd.read_csv(BeerAdvocate_path(data_path)+ "/BeerAdvocate_beers.csv")

In [13]:
extract_nans_as_columns_df(beer_ba, numerical_columns(beer_ba), replace_value=0)
extract_nans_as_columns_df(beer_ba, string_columns(beer_ba), replace_value="")
beer_ba = replace_common_enc_errors(beer_ba)

In [14]:
with open(BeerAdvocate_path(clean_path) + "/beers.csv", "w") as f:
    beer_ba.to_csv(f, index=False)
    
del beer_ba

### Clean BeerAdvocate breweries.csv

In [15]:
brewery_ba = pd.read_csv(BeerAdvocate_path(data_path) + "/BeerAdvocate_breweries.csv")

In [16]:
with open(BeerAdvocate_path(clean_path) + "/breweries.csv", "w") as f:
    brewery_ba.to_csv(f, index=False)

## Clean Ratebeer data

In [17]:
users_rb = pd.read_csv(Ratebeer_path(data_path) + "/Ratebeer_users.csv")

### Clean Ratebeer users.csv

In [18]:
us_users_rb = filter_only_americans(users_rb.dropna(subset=["user_name", "location", "joined"]))
os.makedirs(Ratebeer_path(clean_path), exist_ok=True)
with open(Ratebeer_path(clean_path) + "/usa_users.csv", "w") as f:
    us_users_rb.to_csv(f, index=False)

### Clean Ratebeer ratings.csv

In [19]:
ratings_rb = pd.read_csv(Ratebeer_path(data_path) + "/Ratebeer_ratings.csv")

In [22]:
ratings_rb = ratings_rb.dropna(subset=["user_id", "beer_id"])
ratings_rb = clean_ratings(ratings_rb)
us_ratings_rb = filter_by_users(ratings_rb, us_users_rb)

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[column_name][is_nan] = replace_value
/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183

In [23]:
with open(Ratebeer_path(clean_path) + "/usa_ratings.csv", "w") as f:
    us_ratings_rb.to_csv(f, index=False)
    
del ratings_rb
del us_ratings_rb
del users_rb
del us_users_rb

NameError: name 'us_users_ba' is not defined

### Clean Ratebeer beers.csv

In [25]:
beer_rb = pd.read_csv(Ratebeer_path(data_path) + "/Ratebeer_beers.csv")

In [26]:
extract_nans_as_columns_df(beer_rb, numerical_columns(beer_rb), replace_value=0)
extract_nans_as_columns_df(beer_rb, string_columns(beer_rb), replace_value="")
replace_common_enc_errors(beer_rb)

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[column_name][is_nan] = replace_value
/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183

,Unnamed: 0,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,overall_score,style_score,avg,...,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings,overall_score_nan,style_score_nan,avg_nan,abv_nan,avg_computed_nan,zscore_nan,avg_matched_valid_ratings_nan
0,0,410549,33 Export (Gabon),3198,Sobraga,Pale Lager,1,0.0,0.0,2.72,...,0.000000,0,0.000000,1,1,0,0,0,1,1
1,1,105273,Castel Beer (Gabon),3198,Sobraga,Pale Lager,10,5.0,58.0,2.18,...,0.000000,0,0.000000,0,0,0,0,0,1,1
2,2,19445,Régab,3198,Sobraga,Pale Lager,27,1.0,22.0,1.83,...,0.000000,0,0.000000,0,0,0,0,0,1,1
3,3,155699,Ards Bally Black Stout,13538,Ards Brewing Co.,Stout,6,0.0,0.0,3.18,...,0.823699,6,3.866667,1,1,0,0,0,0,0
4,4,239097,Ards Belfast 366,13538,Ards Brewing Co.,Golden Ale/Blond Ale,1,0.0,0.0,2.79,...,0.000000,0,0.000000,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442076,442076,189684,Stela Selekt,1107,Stefani & Co,Pilsener,5,0.0,0.0,2.19,...,0.000000,0,0.000000,1,1,0,0,0,1,1
442077,442077,84884,Hotel Martini Donauer,9355,Hotel Martini,Pale Lager,1,0.0,0.0,2.77,...,0.000000,0,0.000000,1,1,0,1,0,1,1
442078,442078,93783,Birra Rozafa,9928,Rozafa Brewery,Pale Lager,1,0.0,0.0,2.64,...,0.000000,0,0.000000,1,1,0,1,0,1,1
442079,442079,220897,Svejk Blonde,17155,Svejk Beer Garden,Pale Lager,4,0.0,0.0,2.70,...,0.000000,0,0.000000,1,1,0,1,0,1,1


In [27]:
with open(Ratebeer_path(clean_path) + "/beers.csv", "w") as f:
    beer_rb.to_csv(f, index=False)
    
del beer_rb

### Clean Ratebeer breweries.csv
- No work needed

In [28]:
brewery_rb = pd.read_csv(Ratebeer_path(data_path) + "/Ratebeer_breweries.csv")
with open(Ratebeer_path(clean_path) + "/breweries.csv", "w") as f:
    brewery_rb.to_csv(f, index=False)

## Clean Matched data

In [29]:
users = pd.read_csv(Matched_path(data_path) + "/matched_beer_data_users.csv", low_memory=False)
ratings = pd.read_csv(Matched_path(data_path) + "/matched_beer_data_ratings.csv", low_memory=False)
beers = pd.read_csv(Matched_path(data_path) + "/matched_beer_data_beers.csv", low_memory=False)
breweries = pd.read_csv(Matched_path(data_path) + "/matched_beer_data_breweries.csv", low_memory=False)
users_approx = pd.read_csv(Matched_path(data_path) + "/matched_beer_data_users_approx.csv", low_memory=False)

### Clean Matched users

In [30]:
users = clean_matched_data(users)
us_states = users[users["ba_location"].str.contains(",")]["ba_location"].unique()
us_users = users[users["ba_location"].isin(us_states)]


/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:102: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [31]:
os.makedirs(Matched_path(clean_path), exist_ok=True)
with open(Matched_path(clean_path) + "/usa_users.csv", "w") as f:
    us_users.to_csv(f, index=False)

### Clean Matched users_approx

In [32]:
users_approx = clean_matched_data(users_approx)
us_users_approx = users_approx[users_approx["ba_location"].isin(us_states)]

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:102: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [33]:
with open(Matched_path(clean_path) + "/usa_users_approx.csv", "w") as f:
    us_users_approx.to_csv(f, index=False)

### Clean Matched ratings

In [34]:
ratings = clean_matched_data(ratings)

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:102: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [35]:
# We don't convert str columns to utf-8 because they are already in a good state
extract_nans_as_columns_df(ratings, numerical_columns(ratings), replace_value=0)
extract_nans_as_columns_df(ratings, string_columns(ratings), replace_value="")

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[column_name][is_nan] = replace_value
/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183

In [36]:
with open(Matched_path(clean_path) + "/ratings.csv", "w") as f:
    ratings.to_csv(f, index=False)

### Clean Matched beers

In [37]:
beers = clean_matched_data(beers)

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:102: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [38]:
with open(Matched_path(clean_path) + "/beers.csv", "w") as f:
    beers.to_csv(f, index=False)

### Clean Matched breweries

In [39]:
breweries = clean_matched_data(breweries)

/var/folders/_b/w9s75zbd5tsf70gz6vs4nshh0000gn/T/ipykernel_54915/3965183988.py:102: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [40]:
with open(Matched_path(clean_path) + "/breweries.csv", "w") as f:
    breweries.to_csv(f, index=False)